In [47]:
# over welke periode gaat het dataframe: 10 jaar
# aggregratie op jaar niveau, eventueel later op week niveau

# welke X-en:
# PC4
# LON 
# LAT
# Res Regio
# Gemeente
# tem: hoe warm was het jaar?

# sjv_totaal
# factor_maand
# factor
# week_verbruik_kw
# laadpalen
# electrische auto's
# warmtepompen
# Opwekking zonneenergie
# Opwekking Windenergie
# Decentrale opwekgegevens
# Knelpunt indicatie (Provincie?)

In [48]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [49]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [50]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# Alle benodige ht5 dataframes inlezen

In [116]:
# decentrale_opwekkingsgegevens gegevens inlezen
df_opwek = pd.read_hdf('decentrale_opwekkingsgegevens_data.h5')
# kleinverbruikgegevens gegevens inlezen
df_verbruik = pd.read_hdf('kleinverbruikgegevens_data.h5')
# mapping van CBS buurt naar PC4
df_cbs_buurt_pc4 = pd.read_hdf('cbs_buurt_pc4.h5')
# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')

In [117]:
# 5 CBS data csv files inlezen
cbs_2015 = pd.read_csv('../raw/CBS_PC4_2015_v2.csv')
cbs_2016 = pd.read_csv('../raw/CBS_PC4_2016_v2.csv')  
cbs_2017 = pd.read_csv('../raw/CBS_PC4_2017_v3.csv')  
cbs_2018 = pd.read_csv('../raw/CBS_PC4_2018_v2.csv')  
cbs_2019 = pd.read_csv('../raw/CBS_PC4_2019_v1.csv')  

# Add column JAAR and fill with correct year values. 
cbs_2015['JAAR']=2015
cbs_2016['JAAR']=2016
cbs_2017['JAAR']=2017
cbs_2018['JAAR']=2018
cbs_2019['JAAR']=2019

# Concatenate in a single dataframe.
cbs = pd.concat([cbs_2015,cbs_2016,cbs_2017,cbs_2018,cbs_2019])

# Delet obsolete dataframes
del cbs_2015, cbs_2016, cbs_2017, cbs_2018, cbs_2019

# Nieuw master dataframe maken

In [118]:
# maak nieuw dataframe met de jaren
df = pd.DataFrame(pd.date_range(start='1/1/2010', end='31/12/2020' , freq='Y'), columns=['DATUM']).sort_values(by='DATUM', ascending=False)
df['JAAR'] = df['DATUM'].dt.year

# Vebruik toevoegen aan dataframe

In [119]:
# df_verbruik is nog per pc6, dus eerst omzetten naar pc4
df_verbruik_pc4 = df_verbruik.groupby(['PC4','JAAR']
                                      , as_index=False
                                     ).agg({'SJV_TOTAAL': 'sum'
                                            ,'AANSLUITINGEN_AANTAL': 'sum'  
                                            ,'LEVERINGSRICHTING_PERC': 'mean'
                                           }
                                          )

# join df met df_verbruik
df = pd.merge(df, df_verbruik_pc4, on='JAAR', how='left')

# Opwek toevoegen aan dataframe

In [120]:
# jaar toevoegen
# 2020-07-01 verwijderen, de maand van de peildatum moet januari zijn
df_opwek = df_opwek[df_opwek['Peildatum'].dt.month == 1]
# 2020-01-01 wordt 2019, 2021-01-01 wordt 2020
df_opwek['JAAR'] = df_opwek['Peildatum'].dt.year - 1
# PC4 toevoegen
df_cbs_buurt_pc4 = df_cbs_buurt_pc4.rename(columns={'Buurt2020': 'CBS Buurtcode'})
df_opwek = pd.merge(df_opwek, df_cbs_buurt_pc4, on='CBS Buurtcode', how='left', suffixes=('','_2'))
# df_opwek group by PC4 en JAAR
df_opwek_pc4 = df_opwek.groupby(['PC4','JAAR'], as_index=False).agg({'Gemeente':'max'
                                                                     , 'Aantal aansluitingen in CBS-buurt': 'sum'
                                                                     , 'Aantal aansluitingen met opwekinstallatie':'sum'
                                                                     , 'Opgesteld vermogen':'sum'
                                                                    }
                                                                   )
# rename columns
df_opwek_pc4 = df_opwek_pc4.rename(columns={'Aantal aansluitingen in CBS-buurt': 'AANTAL AANSLUITINGEN IN PC4'
                                            , 'Gemeente': 'GEMEENTE'
                                            , 'Aantal aansluitingen met opwekinstallatie': 'AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE'
                                            , 'Opgesteld vermogen': 'OPGESTELD VERMOGEN'
                                           }
                                  )

# join df met df_opwek_pc4
df = pd.merge(df, df_opwek_pc4, on=['JAAR','PC4'], how='left')

# RES regio toevoegen aan dataframe

In [121]:
df = pd.merge(df, df_pc4_res, on=['PC4'], how='left')

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17130 entries, 0 to 17129
Data columns (total 11 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   DATUM                                      17130 non-null  datetime64[ns]
 1   JAAR                                       17130 non-null  int64         
 2   PC4                                        17130 non-null  object        
 3   SJV_TOTAAL                                 17130 non-null  float64       
 4   AANSLUITINGEN_AANTAL                       17130 non-null  int64         
 5   LEVERINGSRICHTING_PERC                     17130 non-null  float64       
 6   GEMEENTE                                   2436 non-null   object        
 7   AANTAL AANSLUITINGEN IN PC4                2436 non-null   float64       
 8   AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE  2436 non-null   float64       
 9   OPGESTELD VERMOGE

# CBS data voor 2015-2019 toevoegen. Selectie van relevante features nog niet  gedaan¶

In [135]:
df['PC4'] = df['PC4'].astype('int64')

In [137]:
cbs['PC4'].head()

0    1011
1    1012
2    1013
3    1014
4    1015
Name: PC4, dtype: int64

In [107]:
cbs.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
INWONER,20308.0,3619.416831,9000.093396,-99997.0,660.00,2650.00,6910.000,28410.0
MAN,20308.0,1421.189679,8552.518942,-99997.0,340.00,1320.00,3395.000,14195.0
VROUW,20308.0,1371.024769,9057.870561,-99997.0,320.00,1315.00,3490.000,14215.0
INW_014,20308.0,-1821.440467,15758.688512,-99997.0,105.00,410.00,1090.000,6535.0
INW_1524,20308.0,-2122.197016,16125.515560,-99997.0,75.00,300.00,795.000,4905.0
INW_2544,20308.0,-687.121430,13224.500799,-99997.0,130.00,565.00,1650.000,9710.0
INW_4564,20308.0,218.849616,9960.439015,-99997.0,215.00,780.00,1905.000,8860.0
INW_65PL,20308.0,-1181.677221,13993.603006,-99997.0,120.00,485.00,1255.000,5310.0
GEBOORTE,20308.0,-23239.402501,42285.093479,-99997.0,5.00,25.00,65.000,430.0
P_NL_ACHTG,20308.0,-679.450709,8710.302554,-99997.0,80.00,90.00,90.000,100.0


In [138]:
# Change PC4 column datatype to int64 to allow merge
df['PC4'] = df['PC4'].astype('int64')

# Merge cbs data for years 2015-2019. Other years to be loacted or constructed 
df = pd.merge(df, cbs, on=['JAAR','PC4'], how='left')

In [140]:
# Review results
pd.options.display.max_rows = 500
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
JAAR,17130.0,2015.010274,3.168513,2010.000,2012.000000,2015.00000,2018.000000,2020.0
PC4,17130.0,7175.969527,1770.313330,4251.000,5563.000000,7429.00000,8917.000000,9999.0
SJV_TOTAAL,17130.0,7419.743900,7319.597724,26.000,1617.250000,4894.50000,11216.750000,52390.0
AANSLUITINGEN_AANTAL,17130.0,1609.624635,1679.368256,10.000,259.000000,902.50000,2607.000000,10063.0
LEVERINGSRICHTING_PERC,17130.0,93.662165,7.712743,26.895,90.444444,96.42182,99.693618,100.0
AANTAL AANSLUITINGEN IN PC4,2436.0,1866.335796,1773.718166,17.000,418.750000,1212.00000,2913.750000,8897.0
AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE,2436.0,304.234811,301.156217,10.000,81.000000,205.00000,437.250000,2475.0
OPGESTELD VERMOGEN,2436.0,1330.252463,1216.816079,33.000,447.000000,953.50000,1847.500000,8278.0
INWONER,7731.0,3599.881387,4707.024553,-99997.000,600.000000,2185.00000,6017.500000,23005.0
MAN,7731.0,1654.812702,4706.014075,-99997.000,315.000000,1100.00000,2965.000000,11495.0


In [141]:
df[df['JAAR']==2019].head().transpose()

,1578,1579,1580,1581,1582
DATUM,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00
JAAR,2019,2019,2019,2019,2019
PC4,4251,4254,4255,4261,4264
SJV_TOTAAL,22391.0,10609.0,6383.0,12843.0,7366.0
AANSLUITINGEN_AANTAL,4720,2446,1630,2495,1284
LEVERINGSRICHTING_PERC,93.148667,86.7605,84.559487,92.70069,95.191429
GEMEENTE,Altena,NaN,NaN,Altena,Altena
AANTAL AANSLUITINGEN IN PC4,1922.0,NaN,NaN,289.0,1179.0
AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE,117.0,NaN,NaN,39.0,76.0
OPGESTELD VERMOGEN,335.0,NaN,NaN,197.0,383.0


# Save dataframe 

In [10]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [12]:
try:
    os.remove('master_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('master_data.h5')
store['master_data'] = df
store.close()